In [1]:
import sys
sys.path.insert(0, '..')


In [2]:
from IPython.display import Image
%matplotlib inline


In [3]:
import torch
import numpy as np

print('PyTorch version:', torch.__version__)
print('NumPy version: ', np.__version__)


PyTorch version: 2.4.1
NumPy version:  1.24.3


In [4]:
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate(
            [zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, (int((len(x_padded) - len(w_rot))/s) + 1) * s, s):
        res.append(np.sum(
            x_padded[i:i + w_rot.shape[0]] * w_rot))
    return np.array(res)


## Testing:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print('Conv1d Implementation:',
      conv1d(x, w, p=2, s=1))

print('Numpy Results:',
      np.convolve(x, w, mode='same')) 


Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results: [ 5 14 16 26 24 34 19 22]


In [5]:
import scipy.signal


def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1,::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0],
    p[1]:p[1]+X_orig.shape[1]] = X_orig

    res = []
    for i in range(0, (int((X_padded.shape[0] -
                            W_rot.shape[0]) / s[0]) + 1) * s[0], s[0]):
        res.append([])
        for j in range(0, (int((X_padded.shape[1] -
                                W_rot.shape[1]) / s[1]) + 1) * s[1], s[1]):
            X_sub = X_padded[i:i + W_rot.shape[0],
                    j:j + W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print('Conv2d Implementation:\n',
    conv2d(X, W, p=(1, 1), s=(1, 1)))


# The mode='same' parameter in scipy.signal.convolve2d ensures that the output array has the same shape as the input array X. 
# This is achieved by padding the input array appropriately before performing the convolution.
print('SciPy Results (mode="same"):\n',
    scipy.signal.convolve2d(X, W, mode='same'))
""

Conv2d Implementation:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
SciPy Results (mode="same"):
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


''

In [ ]:
import torch.nn as nn  # Import the neural network module from PyTorch
loss_func = nn.BCELoss()  # Initialize the binary cross-entropy loss function
loss = loss_func(torch.tensor([0.9]), torch.tensor([1.0]))  # Calculate loss between predicted (0.9) and target (1.0) values
l2_lambda = 0.001  # Set the L2 regularization strength

conv_layer = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5)  # Create a 2D convolutional layer with 3 input channels and 5 output channels
l2_penalty = l2_lambda * sum([(p**2).sum() for p in conv_layer.parameters()])  # Compute L2 penalty for convolutional layer parameters
loss_with_penalty = loss + l2_penalty  # Add L2 penalty to the original loss

linear_layer = nn.Linear(10, 16)  # Create a linear layer with 10 input features and 16 output features
l2_penalty = l2_lambda * sum([(p**2).sum() for p in linear_layer.parameters()])  # Compute L2 penalty for linear layer parameters
loss_with_penalty = loss + l2_penalty  # Add L2 penalty to the original loss again
